In [7]:
import tensorflow as tf
import pandas as pd

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
def my_model(features, labels, mode, params):
    # 网络结构
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [8]:
def load_data(y_name='Species'):
    train_path = '../data/iris_training.csv'
    test_path = '../data/iris_test.csv'
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)
    
    return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = load_data()
my_feature_columns = []
for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [10]:
def train_input_fn(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [9]:
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    model_dir = './custom_logs',
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [10, 10],
        # The model must choose between 3 classes.
        'n_classes': 3,
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_evaluation_master': '', '_task_type': 'worker', '_task_id': 0, '_master': '', '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './custom_logs', '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000309BEF3BE0>, '_service': None, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_is_chief': True}


In [13]:
classifier.train(
        input_fn=lambda:train_input_fn(train_x, train_y, 100),
        steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./custom_logs\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into ./custom_logs\model.ckpt.
INFO:tensorflow:loss = 0.06206132, step = 1000
INFO:tensorflow:global_step/sec: 399.736
INFO:tensorflow:loss = 0.024732469, step = 1100 (0.254 sec)
INFO:tensorflow:global_step/sec: 514.492
INFO:tensorflow:loss = 0.051306788, step = 1200 (0.192 sec)
INFO:tensorflow:global_step/sec: 562.736
INFO:tensorflow:loss = 0.05293573, step = 1300 (0.178 sec)
INFO:tensorflow:global_step/sec: 625.242
INFO:tensorflow:loss = 0.07312765, step = 1400 (0.159 sec)
INFO:tensorflow:global_step/sec: 628.329
INFO:tensorflow:loss = 0.03292082, step = 1500 (0.160 sec)
INFO:tensorflow:global_step/sec: 630.307
INFO:tensorflow:loss = 0.0

In [19]:
help(classifier.predict)

Help on method predict in module tensorflow.python.estimator.estimator:

predict(input_fn, predict_keys=None, hooks=None, checkpoint_path=None, yield_single_examples=True) method of tensorflow.python.estimator.estimator.Estimator instance
    Yields predictions for given features.
    
    Args:
      input_fn: A function that constructs the features. Prediction continues
        until `input_fn` raises an end-of-input exception (`OutOfRangeError` or
        `StopIteration`).
        See @{$get_started/premade_estimators#create_input_functions} for more
        information. The function should construct and return one of
        the following:
    
          * A 'tf.data.Dataset' object: Outputs of `Dataset` object must have
            same constraints as below.
          * features: A `Tensor` or a dictionary of string feature name to
            `Tensor`. features are consumed by `model_fn`. They should satisfy
            the expectation of `model_fn` from inputs.
          * A tup

In [14]:
dir(tf.estimator.EstimatorSpec)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 '_source',
 '_tf_api_names',
 'count',
 'eval_metric_ops',
 'evaluation_hooks',
 'export_outputs',
 'index',
 'loss',
 'mode',
 'prediction_hooks',
 'predictions',
 'scaffold',
 'train_op',
 'training_chief_hooks',
 'training_hooks']